##  Import Library

In [ ]:
#import package library
from sklearn.cluster import KMeans
import numpy as np
import pandas as pd
# some imports to set up plotting 
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline

##  Exploratory Data Analysis 

โหลดไฟล์ CSV ขึ้นมาใส่ Pandas Dataframe แล้วลองวิเคราะห์สำรวจข้อมูล Exploratory Data 

In [ ]:
df =  pd.read_csv('Mall_Customers.csv')
df.shape

ดูข้อมูลตัวอย่าง 5 แถวแรก

In [ ]:
df.head()

**ดูสถิติข้อมูล**

In [ ]:
df.describe()

ดูชนิดข้อมูล เป็นตัวเลข ยกเว้น Gender ที่เป็น Object (String)

In [ ]:
df.dtypes

In [ ]:
df.isnull().sum()

### ดู Histogram ของทั้ง 3 Column ที่เป็นตัวเลข

ดูการการะจายข้อมูลของทั้ง 3 Column

In [ ]:
plt.figure(1 , figsize = (15 , 6))
n = 0 
for x in ['Age' , 'Annual Income (k$)' , 'Spending Score (1-100)']:
    n += 1
    plt.subplot(1 , 3 , n)
    plt.subplots_adjust(hspace =0.5 , wspace = 0.5)
    sns.distplot(df[x] , bins = 20)
    plt.title('Distplot of {}'.format(x))
plt.show()

### สัดส่วน แต่ละเพศ

นับจำนวนตัวอย่างข้อมูล แยก Gender

In [ ]:
labels = ['Female', 'Male']
size = df['Gender'].value_counts()
size

In [ ]:
plt.rcParams['figure.figsize'] = (7, 7)
plt.pie(size,labels = labels, autopct = '%.2f%%')
plt.title('Gender', fontsize = 20)
plt.axis('off')
plt.legend()
plt.show()

### พล็อตกราฟความสัมพันธ์ ระหว่าง Age, Annual Income, Spending Score

พล็อตข้อมูล และ fit ด้วย Linear Regression โดยยังไม่สนใจเพศ Gender

In [ ]:
plt.figure(1 , figsize = (10 , 7))
n = 0 
for x in ['Age' , 'Annual Income (k$)' , 'Spending Score (1-100)']:
    for y in ['Age' , 'Annual Income (k$)' , 'Spending Score (1-100)']:
        n += 1
        plt.subplot(3 , 3 , n)
        plt.subplots_adjust(hspace = 0.5 , wspace = 0.5)
        sns.regplot(x = x , y = y , data = df)
        plt.ylabel(y.split()[0]+' '+y.split()[1] if len(y.split()) > 1 else y )
plt.show()

In [ ]:
#เปรียบเทียบ อายุ เพศ
plt.figure(figsize=(8,4))
sns.distplot( df[df["Gender"]=="Male"].Age, color="skyblue",label="Male")
sns.distplot( df[df["Gender"]=="Female"].Age, color="Red",label="Female")
plt.legend();

In [ ]:
#เปรียบเทียบ รายได้ เพศ
plt.figure(figsize=(8,4))
sns.distplot( df[df["Gender"]=="Male"]["Annual Income (k$)"], color="skyblue",label="Male")
sns.distplot( df[df["Gender"]=="Female"]["Annual Income (k$)"], color="Red",label="Female")
plt.legend();

In [ ]:
#เปรียบเทียบ คะแนนการใช้จ่าย เพศ
plt.figure(figsize=(8,4))
sns.distplot( df[df["Gender"]=="Male"]["Spending Score (1-100)"], color="skyblue",label="Male")
sns.distplot( df[df["Gender"]=="Female"]["Spending Score (1-100)"], color="Red",label="Female")
plt.legend();

##แปลง Gender เป็นตัวเลข

In [ ]:
gender = {'Male': 0,'Female': 1} 
df.Gender = [gender[item] for item in df.Gender] 

In [ ]:
df.head()

### Heatmap ของ Correlation 

In [ ]:
plt.rcParams['figure.figsize'] = (12, 8)
sns.heatmap(df.drop("CustomerID", axis=1).corr(), cmap = 'BuPu', annot = True)
plt.title('Heatmap for the Data', fontsize = 20)
plt.show()

สำหรับ kMean ข้อมูลที่ใช้จำเป็นต้องเป็น numerical

In [ ]:
X = df.drop(["Gender","CustomerID"], axis=1) #คำถาม: ทำไมถึงตัด CustomerID ทิ้ง?
X.head()

สังเกตว่าข้อมูลนั้นไม่ได้อยู่ใน scale เดียวกัน จึงจำเป็นต้องทำการปรับให้ใกล้เคียงกันก่อน

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_standard=scaler.fit_transform(X)

In [ ]:
from sklearn.cluster import KMeans
km = KMeans(n_clusters=3)
km.fit(X_standard)
label = km.predict(X_standard)
centers_standard = km.cluster_centers_

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
import ipywidgets as widgets
from IPython.display import display

In [ ]:
fig = plt.figure(1, figsize=(6,6))
ax = fig.add_subplot(projection='3d')
ax.scatter(X_standard[:, 0], X_standard[:, 1], X_standard[:, 2], c=label, s=30)
ax.scatter (centers_standard[:,0],centers_standard[:,1],centers_standard[:,2], c='black',s=200)
ax.set_title("k-Mean Results Standardized")
ax.set_xlabel("Age")
ax.set_ylabel("Annual Income")
ax.set_zlabel("Spending Score")

plt.show()

เปลี่ยนข้อมูลกลับไป scale เดิม

In [ ]:
centers_original = scaler.inverse_transform(centers_standard)
X_original = scaler.inverse_transform(X_standard) # จริงๆใช้ค่า X ตั้งต้นก็ได้

In [ ]:
pd.DataFrame(centers_original, columns=X.columns) #ใส่ชื่อ column เดิมดู

In [ ]:
fig = plt.figure(1, figsize=(6,6))
ax = fig.add_subplot(projection='3d')
ax.scatter(X_original[:, 0], X_original[:, 1], X_original[:, 2], c=label, s=30)
ax.scatter (centers_original[:,0],centers_original[:,1],centers_original[:,2], c='black',s=200)
ax.set_title("k-Mean Results")
ax.set_xlabel("Age")
ax.set_ylabel("Annual Income")
ax.set_zlabel("Spending Score")

plt.show()

----
## Choosing proper *k*

ทดลอง cluster ด้วยค่า k หลายๆค่าและเลือกอันที่ inertia เหมาะสมที่สุด (ที่ **"elbow"**)

In [ ]:
inertia_list = []
for i in range(1, 11):
    kmeans = KMeans(n_clusters = i, random_state = 42)
    kmeans.fit(X_standard)
    # inertia method returns wcss for that model
    inertia_list.append(kmeans.inertia_)

In [ ]:
plt.plot(inertia_list,marker='o',color='red')
plt.xticks(range(0,10), [str(i) for i in range (1,11)])
plt.xlabel("Number of clusters (k)")
plt.ylabel("Inertia")

จะเห็นได้ว่าค่าของ k=2,4 เป็นอีกตัวเลือกหนึ่งที่น่าสนใจ

In [ ]:
km2 = KMeans(n_clusters=4)
km2.fit(X_standard)
label2 = km2.predict(X_standard)
centers_standard2 = km2.cluster_centers_

In [ ]:
centers_original2 = scaler.inverse_transform(centers_standard2)

In [ ]:
pd.DataFrame(centers_original2, columns=X.columns) #ใส่ชื่อ column เดิมดู

Plot ดู

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
import ipywidgets as widgets
from IPython.display import display

In [ ]:
%matplotlib notebook
fig = plt.figure(2, figsize=(6,6))
ax = fig.add_subplot(projection='3d')
ax.scatter(X_original[:, 0], X_original[:, 1], X_original[:, 2], c=label2, s=30)
ax.scatter (centers_original2[:,0],centers_original2[:,1],centers_original2[:,2], c='black',s=200)
ax.set_title("k-Mean Results")
ax.set_xlabel("Age")
ax.set_ylabel("Annual Income")
ax.set_zlabel("Spending Score")
ax.w_zaxis.set_ticklabels([])

plt.show()

def update_plot(angle1 = 20, angle2 = 40):
    # set view angle
    ax.view_init(angle1, angle2)
    fig.canvas.draw_idle()

# prepare widgets
angle1_slider = widgets.IntSlider(20, min = 0, max = 60)
angle1_label = widgets.Label(value = 'Angle 1 value is: ' + str(angle1_slider.value))
display(angle1_slider, angle1_label)

# handle angle 1 update
def update_angle1(value):
    update_plot(angle2 = value['new'])
    angle1_label.value = 'Angle 2 value is: ' + str(value.new)

angle1_slider.observe(update_angle1, names = 'value')

*Code ของ widget นี้ปรับมาจาก (https://stackoverflow.com/questions/42998009/clear-matplotlib-figure-in-jupyter-python-notebook)*

----
## Silouette Score

เพื่อช่วยในการตัดสินใจ เราอาจนำ silouette score เข้ามาช่วยพิจารณาด้วย 
*(code ของ plot ปรับจาก https://scikit-learn.org/stable/modules/generated/sklearn.metrics.silhouette_score.html)*

In [ ]:
%matplotlib inline
from sklearn.metrics import silhouette_samples, silhouette_score
import matplotlib.cm as cm

range_n_clusters = range(2,10)

for n_clusters in range_n_clusters:
    # Create a subplot with 1 row and 2 columns
    fig, ax1 = plt.subplots(1, 1)
    fig.set_size_inches(18, 7)

    # The 1st subplot is the silhouette plot
    # The silhouette coefficient can range from -1, 1 but in this example all
    # lie within [-0.1, 1]
    ax1.set_xlim([-0.1, 1])
    # The (n_clusters+1)*10 is for inserting blank space between silhouette
    # plots of individual clusters, to demarcate them clearly.
    ax1.set_ylim([0, len(X_standard) + (n_clusters + 1) * 10])

    # Initialize the clusterer with n_clusters value and a random generator
    # seed of 10 for reproducibility.
    clusterer = KMeans(n_clusters=n_clusters, random_state=10)
    cluster_labels = clusterer.fit_predict(X_standard)

    # The silhouette_score gives the average value for all the samples.
    # This gives a perspective into the density and separation of the formed
    # clusters
    silhouette_avg = silhouette_score(X_standard, cluster_labels)
    print("For n_clusters =", n_clusters,
          "The average silhouette_score is :", silhouette_avg)

    # Compute the silhouette scores for each sample
    sample_silhouette_values = silhouette_samples(X_standard, cluster_labels)

    y_lower = 10
    for i in range(n_clusters):
        # Aggregate the silhouette scores for samples belonging to
        # cluster i, and sort them
        ith_cluster_silhouette_values = \
            sample_silhouette_values[cluster_labels == i]

        ith_cluster_silhouette_values.sort()

        size_cluster_i = ith_cluster_silhouette_values.shape[0]
        y_upper = y_lower + size_cluster_i

        color = cm.nipy_spectral(float(i) / n_clusters)
        ax1.fill_betweenx(np.arange(y_lower, y_upper),
                          0, ith_cluster_silhouette_values,
                          facecolor=color, edgecolor=color, alpha=0.7)

        # Label the silhouette plots with their cluster numbers at the middle
        ax1.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))

        # Compute the new y_lower for next plot
        y_lower = y_upper + 10  # 10 for the 0 samples

    ax1.set_title("The silhouette plot for the various clusters.")
    ax1.set_xlabel("The silhouette coefficient values")
    ax1.set_ylabel("Cluster label")

    # The vertical line for average silhouette score of all the values
    ax1.axvline(x=silhouette_avg, color="red", linestyle="--")

    ax1.set_yticks([])  # Clear the yaxis labels / ticks
    ax1.set_xticks([-0.1, 0, 0.2, 0.4, 0.6, 0.8, 1])


plt.show()